In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[1]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,9484.0,0.513492,-0.046,0.0,0.0
1,9152.0,0.474906,-0.302,0.0,0.0
2,8799.0,0.433880,-0.541,0.0,0.0
3,8567.0,0.406916,-0.794,0.0,0.0
4,8487.0,0.397618,-1.053,0.0,0.0
...,...,...,...,...,...
43819,9369.0,0.500127,1.684,0.0,0.0
43820,9105.0,0.469444,1.718,0.0,0.0
43821,9346.0,0.497454,1.726,0.0,0.0
43822,9392.0,0.502800,1.569,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[1]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_BE,Temp_BE
0,9484.0,-0.046
1,9152.0,-0.302
2,8799.0,-0.541
3,8567.0,-0.794
4,8487.0,-1.053
...,...,...
43819,9369.0,1.684
43820,9105.0,1.718
43821,9346.0,1.726
43822,9392.0,1.569


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_BE,Temp_BE,Lag-2,Lag-7
0,10519.0,1.499,10657.0,9484.0
1,10083.0,1.842,10281.0,9152.0
2,9803.0,2.338,10054.0,8799.0
3,9828.0,2.934,10078.0,8567.0
4,10238.0,3.446,10415.0,8487.0
...,...,...,...,...
43651,9369.0,1.684,9420.0,8654.0
43652,9105.0,1.718,9091.0,8253.0
43653,9346.0,1.726,9067.0,8638.0
43654,9392.0,1.569,8991.0,8691.0


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 24s 18ms/step - loss: 0.0188 - mape: 7311.1582 - mae: 0.1099 - val_loss: 0.0140 - val_mape: 91.1797 - val_mae: 0.0958
Epoch 2/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0122 - mape: 6983.5488 - mae: 0.0861 - val_loss: 0.0123 - val_mape: 101.1405 - val_mae: 0.0830
Epoch 3/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0096 - mape: 5819.0830 - mae: 0.0741 - val_loss: 0.0090 - val_mape: 71.5503 - val_mae: 0.0743
Epoch 4/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0083 - mape: 5697.7295 - mae: 0.0682 - val_loss: 0.0114 - val_mape: 60.5966 - val_mae: 0.0852
Epoch 5/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0074 - mape: 5409.2866 - mae: 0.0639 - val_loss: 0.0119 - val_mape: 65.6638 - val_mae: 0.0881
Epoch 6/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0069 - mape: 4933.3667 - mae: 0.0612 - val_

1105/1105 [==============================] - 19s 17ms/step - loss: 0.0027 - mape: 2450.4048 - mae: 0.0383 - val_loss: 0.0105 - val_mape: 42.3609 - val_mae: 0.0774
Epoch 48/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0025 - mape: 2331.8103 - mae: 0.0371 - val_loss: 0.0098 - val_mape: 38.8354 - val_mae: 0.0740
Epoch 49/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0025 - mape: 2273.8662 - mae: 0.0369 - val_loss: 0.0101 - val_mape: 50.5770 - val_mae: 0.0758
Epoch 50/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0025 - mape: 2710.2378 - mae: 0.0366 - val_loss: 0.0117 - val_mape: 50.6343 - val_mae: 0.0804
Epoch 51/100
1105/1105 [==============================] - 18s 17ms/step - loss: 0.0025 - mape: 3072.2441 - mae: 0.0366 - val_loss: 0.0100 - val_mape: 56.3937 - val_mae: 0.0747
Epoch 52/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0024 - mape: 2517.7559 - mae: 0.0365 - val_loss: 0.

1105/1105 [==============================] - 19s 17ms/step - loss: 0.0017 - mape: 1709.2522 - mae: 0.0305 - val_loss: 0.0116 - val_mape: 36.3139 - val_mae: 0.0828
Epoch 94/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0013 - mape: 1560.3008 - mae: 0.0277 - val_loss: 0.0112 - val_mape: 40.6822 - val_mae: 0.0793
Epoch 95/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0013 - mape: 1318.5469 - mae: 0.0270 - val_loss: 0.0104 - val_mape: 43.6396 - val_mae: 0.0775
Epoch 96/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0014 - mape: 1407.3132 - mae: 0.0279 - val_loss: 0.0107 - val_mape: 33.1689 - val_mae: 0.0794
Epoch 97/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0013 - mape: 1206.4934 - mae: 0.0275 - val_loss: 0.0110 - val_mape: 56.8655 - val_mae: 0.0789
Epoch 98/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0014 - mape: 1757.6151 - mae: 0.0280 - val_loss: 0.

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1916.5259385108948


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 11ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.21539026, 0.1963035 , 0.22169524, ..., 0.44300312, 0.38409305,
       0.3189086 ], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.29823529, 0.287744  , 0.28825577, ..., 0.44677655, 0.45266191,
       0.41133647])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,8185.00,7537.482422
1,8103.00,7388.300293
2,8107.00,7586.762207
3,8244.26,8030.872070
4,8819.67,8846.067383
...,...,...
4363,9369.00,9371.152344
4364,9105.00,9372.593750
4365,9346.00,9316.506836
4366,9392.00,8856.065430


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

4.950886941507138